# Importing libraries

- pandas  for working with tables (DataFrames)
- numpy  for generating random data efficiently

In [1]:
import pandas as pd
import numpy as np

- Generating the number of rows

In [2]:
N = 500_000  

In [3]:
# Setting a seed so that results are reproducible
np.random.seed(42)

# Creating the dataset using a pandas DataFrame

In [9]:
data = pd.DataFrame({

    # Unique ID for each transaction
    "transaction_id": np.arange(1, N + 1),

    # Generate random dates within one year
    "date": pd.to_datetime("2023-01-01") + pd.to_timedelta(
        np.random.randint(0, 365, N), unit="D"
    ),

    # Randomly assign regions
    "region": np.random.choice(
        ["Central", "Eastern", "Western", "Northern"], N
    ),

    # Random product categories
    "product_category": np.random.choice(
        ["Food", "Electronics", "Clothing", "Furniture"], N
    ),

    # Quantity of items bought (1 to 9)
    "quantity": np.random.randint(1, 10, N),

    # Unit price between 5 and 500
    "unit_price": np.round(np.random.uniform(5, 500, N), 2),

    # Payment methods
    "payment_method": np.random.choice(
        ["Cash", "Mobile Money", "Card"], N
    ),

    # Customer type
    "customer_type": np.random.choice(
        ["Retail", "Wholesale"], N
    )
})


- Creating a derived column for total transaction amount


In [10]:
!pip3 install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 943.1 kB/s  0:01:43m0:00:0100:02

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3.12 -m pip install --upgrade pip


In [11]:
# total_amount = quantity * unit_price
data["total_amount"] = data["quantity"] * data["unit_price"]

# Save the dataset as a CSV file (row-based storage)
data.to_csv("transactions.csv", index=False)

# Save the dataset as a Parquet file (column-based storage)
data.to_parquet("transactions.parquet")

print("Dataset created successfully!")

ArrowKeyError: No type extension with name arrow.py_extension_type found